In [1]:
import polars as pl
import turtle_island as ti

from great_tables import GT, style, loc

## ti.make_index()

In [2]:
df1 = pl.DataFrame({"a": [1, 3, 5], "b": [2, 4, 6]})
print(df1)

shape: (3, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 1   ┆ 2   │
│ 3   ┆ 4   │
│ 5   ┆ 6   │
└─────┴─────┘


In [3]:
print(df1.select(ti.make_index(), pl.all()))

shape: (3, 3)
┌───────┬─────┬─────┐
│ index ┆ a   ┆ b   │
│ ---   ┆ --- ┆ --- │
│ u32   ┆ i64 ┆ i64 │
╞═══════╪═════╪═════╡
│ 0     ┆ 1   ┆ 2   │
│ 1     ┆ 3   ┆ 4   │
│ 2     ┆ 5   ┆ 6   │
└───────┴─────┴─────┘


## ti.case_when()

In [4]:
df2 = pl.DataFrame({"x": [1, 2, 3]})
print(df2)

shape: (3, 1)
┌─────┐
│ x   │
│ --- │
│ i64 │
╞═════╡
│ 1   │
│ 2   │
│ 3   │
└─────┘


In [5]:
print(
    df2.with_columns(
        pl.when(pl.col("x").le(1))
        .then(pl.lit("small"))
        .when(pl.col("x").ge(3))
        .then(pl.lit("large"))
        .otherwise(pl.lit("medium"))
        .alias("y")
    )
)

shape: (3, 2)
┌─────┬────────┐
│ x   ┆ y      │
│ --- ┆ ---    │
│ i64 ┆ str    │
╞═════╪════════╡
│ 1   ┆ small  │
│ 2   ┆ medium │
│ 3   ┆ large  │
└─────┴────────┘


In [6]:
print(
    df2.with_columns(
        ti.case_when(
            case_list=[
                (pl.col("x").le(1), pl.lit("small")),
                (pl.col("x").ge(3), pl.lit("large")),
            ],
            otherwise=pl.lit("medium"),
        ).alias("y")
    )
)

shape: (3, 2)
┌─────┬────────┐
│ x   ┆ y      │
│ --- ┆ ---    │
│ i64 ┆ str    │
╞═════╪════════╡
│ 1   ┆ small  │
│ 2   ┆ medium │
│ 3   ┆ large  │
└─────┴────────┘


## ti.bucketize()

In [7]:
df3 = pl.DataFrame({"x": [1, 2, 3, 4, 5]})
print(df3)

shape: (5, 1)
┌─────┐
│ x   │
│ --- │
│ i64 │
╞═════╡
│ 1   │
│ 2   │
│ 3   │
│ 4   │
│ 5   │
└─────┘


In [8]:
print(
    df3.with_columns(
        ti.bucketize(pl.col("x").add(10), pl.lit(100)).alias("bucketized")
    )
)

shape: (5, 2)
┌─────┬────────────┐
│ x   ┆ bucketized │
│ --- ┆ ---        │
│ i64 ┆ i64        │
╞═════╪════════════╡
│ 1   ┆ 11         │
│ 2   ┆ 100        │
│ 3   ┆ 13         │
│ 4   ┆ 100        │
│ 5   ┆ 15         │
└─────┴────────────┘


## ti.is_every_nth_row()

In [9]:
df4 = pl.DataFrame({"x": [1, 2, 3, 4, 5, 6, 7]})
print(df4)

shape: (7, 1)
┌─────┐
│ x   │
│ --- │
│ i64 │
╞═════╡
│ 1   │
│ 2   │
│ 3   │
│ 4   │
│ 5   │
│ 6   │
│ 7   │
└─────┘


In [10]:
print(df4.with_columns(ti.is_every_nth_row(2)))

shape: (7, 2)
┌─────┬──────────────┐
│ x   ┆ bool_nth_row │
│ --- ┆ ---          │
│ i64 ┆ bool         │
╞═════╪══════════════╡
│ 1   ┆ true         │
│ 2   ┆ false        │
│ 3   ┆ true         │
│ 4   ┆ false        │
│ 5   ┆ true         │
│ 6   ┆ false        │
│ 7   ┆ true         │
└─────┴──────────────┘


### Great Tables

In [11]:
# https://docs.pola.rs/user-guide/expressions/window-functions/
types = (
    "Grass Water Fire Normal Ground Electric Psychic Fighting Bug Steel "
    "Flying Dragon Dark Ghost Poison Rock Ice Fairy".split()
)

url = (
    "https://gist.githubusercontent.com/ritchie46/"
    "cac6b337ea52281aa23c049250a4ff03/"
    "raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/"
    "pokemon.csv"
)

type_enum = pl.Enum(types)

pokemon = (
    pl.read_csv(url)
    .cast({"Type 1": type_enum})
    .sort("Type 1")
    .select(
        pl.col("Type 1")
        .head(3)
        .over("Type 1", mapping_strategy="explode"),
        pl.col("Name")
        .sort_by(pl.col("Speed"), descending=True)
        .head(3)
        .over("Type 1", mapping_strategy="explode")
        .alias("fastest/group"),
        pl.col("Name")
        .sort_by(pl.col("Attack"), descending=True)
        .head(3)
        .over("Type 1", mapping_strategy="explode")
        .alias("strongest/group"),
    )
    .head(9)
)
print(pokemon)

shape: (9, 3)
┌────────┬───────────────────────────┬───────────────────────────┐
│ Type 1 ┆ fastest/group             ┆ strongest/group           │
│ ---    ┆ ---                       ┆ ---                       │
│ enum   ┆ str                       ┆ str                       │
╞════════╪═══════════════════════════╪═══════════════════════════╡
│ Grass  ┆ Venusaur                  ┆ Victreebel                │
│ Grass  ┆ VenusaurMega Venusaur     ┆ VenusaurMega Venusaur     │
│ Grass  ┆ Victreebel                ┆ Exeggutor                 │
│ Water  ┆ Starmie                   ┆ GyaradosMega Gyarados     │
│ Water  ┆ Tentacruel                ┆ Kingler                   │
│ Water  ┆ Poliwag                   ┆ Gyarados                  │
│ Fire   ┆ Rapidash                  ┆ CharizardMega Charizard X │
│ Fire   ┆ Charizard                 ┆ Flareon                   │
│ Fire   ┆ CharizardMega Charizard X ┆ Arcanine                  │
└────────┴───────────────────────────┴──────────

In [12]:
row_expr = ti.is_every_nth_row(3)

(
    GT(pokemon)
    .tab_style(
        style=style.fill("lightblue"), locations=loc.body(rows=row_expr)
    )
    .tab_style(
        style=style.fill("papayawhip"), locations=loc.body(rows=~row_expr)
    )
    .opt_stylize(style=3, color="pink")
)

Type 1,fastest/group,strongest/group
Grass,Venusaur,Victreebel
Grass,VenusaurMega Venusaur,VenusaurMega Venusaur
Grass,Victreebel,Exeggutor
Water,Starmie,GyaradosMega Gyarados
Water,Tentacruel,Kingler
Water,Poliwag,Gyarados
Fire,Rapidash,CharizardMega Charizard X
Fire,Charizard,Flareon
Fire,CharizardMega Charizard X,Arcanine
